# utils

> Helper functions for estout package

In [ ]:
#|default_exp utils

In [ ]:
#|hide 
from nbdev.showdoc import *
from nbdev.test import *

In [ ]:
#|export
from __future__ import annotations
import os, platform, functools
from pathlib import Path
from subprocess import PIPE, Popen, run, SubprocessError
from typing import Dict, List, Tuple, Literal 
import pandas as pd 
import numpy as np

In [ ]:
#| export
def rgetattr(obj, attr, *args):
    """Recursive getattr (for nested attributes)."""
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))

In [ ]:
#| export
def rsetattr(obj, attr, val):
    """Recursive setattr (for nested attributes)."""
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

In [ ]:
#| export
def default_formats() -> dict:
    """Default output formats for some very common statistics."""
    return  {'params':'{:.2f}', 
            'tstats':'{:.2f}', 
            'pvalues': '{:.3f}',
            'se':'{:.2f}', 
            'r2':'{:.3f}',
            'nobs':'{:.0f}'
            }

In [ ]:
default_formats()

{'params': '{:.2f}',
 'tstats': '{:.2f}',
 'pvalues': '{:.3f}',
 'se': '{:.2f}',
 'r2': '{:.3f}',
 'nobs': '{:.0f}'}

In [ ]:
#| export 
def get_stars(pvalues: pd.Series, # This is compared to key of 'stars' parameter to determine how many stars should be added
            stars: dict = {.1:'*',.05:'**',.01:'***'} # Todo: default values to the left are star symbols that are not rendered correctly in markdown
            ) -> pd.Series:
    """For each pvalue, check the lowest key in 'stars' for which the pvalue is smaller than that key, and return the corresponding nr of stars."""

    param_names = list(pvalues.index)

    #Sort 'stars' by key (in reverse order)
    ks = list(stars.keys())
    ks.sort(reverse=True)
    stars = {k: stars[k] for k in ks}

    out = pd.Series('', index=param_names)
    for param in param_names:    
        for alpha in stars:
            if pvalues[param] < alpha:
                out[param] = stars[alpha]
                
    return out

In [ ]:
get_stars(pvalues= pd.Series([0.5, .03, 0.002], index=['x1','x2','x3']))

x1       
x2     **
x3    ***
dtype: object

In [ ]:
get_stars(pvalues= pd.Series([0.5, .03, 0.002], index=['x1','x2','x3']), 
          stars={0.4: '**', 0.9: '***'})

x1    ***
x2     **
x3     **
dtype: object

In [ ]:
#| export
def model_groups(column_group_names: Dict[str, List[int]], # Keys are group titles, values are lists of column indices included in each group
                add_clines: bool=True # If True, adds lines below group names
                ) -> str:
    """Returns LaTex code needed to add at the top of the table in order to give names to groups of columns in the table."""
    
    group_names = ''
    group_lines = ''
    for key,value in column_group_names.items():
        if type(key) != str:
            raise TypeError('Each key in column_group_names must be a string')
        if type(value) != list:
            raise TypeError("Each value in column_group_names dict must be a list")
        if len(value) != 2:
            raise TypeError("Each value in column_group_names dict must contain two integers")
        
        value = sorted(value)
        group_names += '& \multicolumn{%i}{c}{%s} ' %(value[1]-value[0]+1,key)
        if add_clines:
            group_lines += '\cline{%s-%s} ' %(str(value[0]+1),str(value[1]+1))

    return group_names + ' \\\\ \n' + group_lines + ' \n'

In [ ]:
print(model_groups(column_group_names={'Group 1':[0,1], 'Group 2':[2,3]}))

& \multicolumn{2}{c}{Group 1} & \multicolumn{2}{c}{Group 2}  \\ 
\cline{1-2} \cline{3-4}  



In [ ]:
#| export
def tex_table_env(nr_columns: int, # Number of columns in the table
                    env: Literal['tabularx','tabular*']='tabular*' # Latex tabular environment specification
                    ) -> Tuple[str,str]:
    """Creates LaTex code to add at the top of the table to create the correct tabular environment."""

    if env=='tabularx':
        header = '\\begin{tabularx}{\\textwidth}{@{}l *{%i}{>{\centering\\arraybackslash}X}@{}}' %nr_columns 
        footer = '\\end{tabularx}'
    elif env=='tabular*':
        header = '\\begin{tabular*}{\\textwidth}{@{\extracolsep{\\fill}}l*{%i}{c}}' %nr_columns
        footer = '\\end{tabular*}'
    else:
        raise NotImplemented(f"LaTex tabular environment {env} has not yet been implemented in tex_table_env()")
    return header,footer 

In [ ]:
header,footer = tex_table_env(nr_columns=4, env='tabularx')
print(header)
print(footer)

\begin{tabularx}{\textwidth}{@{}l *{4}{>{\centering\arraybackslash}X}@{}}
\end{tabularx}


In [ ]:
header,footer = tex_table_env(nr_columns=4, env='tabular*')
print(header)
print(footer)

\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}l*{4}{c}}
\end{tabular*}


In [ ]:
#|export
def df_to_tex(df: pd.DataFrame, # If this has a multiindex, only the first level is kept
                panel_title: str='',
                palign: Literal['l','r','c']='l', # Alignment of panel title 
                col_groups: Dict[str, List[int]]=None, # Keys are group names; values are lists of consecutive indices of columns in the group
                col_names: List[str]|bool=True, # If False, none; if True, use df column names; if list, gives custom column names
                hlines: List[int]=[], # Row indices under which to place hline (use 0 for topline). adds \smallskip under each line
                tabular_env: str='tabular*' #LaTex tabular environment
                ) -> str: 
    """Creates LaTeX-formatted table from DataFrame."""

    nr_cols = len(df.columns)
    df = df.astype('string')
    if isinstance(df.index, pd.MultiIndex): df = df.droplevel(1)

    header,footer = tex_table_env(nr_cols, tabular_env)
    out = header + ' \n'

    if panel_title:
        out += f'\multicolumn{{{nr_cols+1}}}{{@{{}} {palign}}}{{{panel_title}}}' + ' \\\\ \n'

    if col_groups: out += model_groups(col_groups)
    if col_names is True: col_names = [str(x) for x in df.columns]
    if col_names: out += ' & '.join([''] + col_names) + ' \\\\ \n'
    for rownr in range(df.shape[0]):
        out += str(df.index[rownr]) + ' & '  + ' & '.join(list(df.iloc[rownr])) +  ' \\\\ \n' 
    out = out + footer

    linebreaks = [i for i in range(len(out)) if out[i:].startswith('\n')]
    for line_nr, pos in reversed(list(enumerate(linebreaks))):
        if line_nr in hlines: out = out[:pos+1] + ' \hline \\noalign{\smallskip} \n' + out[pos+1:]

    return out

Panel title (if any) will appear first, then column group names(if any), then column names (if any), then the body of the table.


In [ ]:
df = pd.DataFrame(np.random.rand(5,4), index=['Var1','','Var2','','R2'], columns=list('ABCD'))
df

,A,B,C,D
Var1,0.988981,0.598698,0.702370,0.442639
,0.620074,0.983868,0.726391,0.261430
Var2,0.781038,0.537924,0.275638,0.017764
,0.331525,0.288122,0.180170,0.966439
R2,0.504925,0.832803,0.681793,0.566745


In [ ]:
tex_string = df_to_tex(df, panel_title='Dependent variable is Y',
                        col_groups={'Group 1':[1,2], 'Group 2': [3,4]},
                        hlines=[0,2,7])
print(tex_string)

\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}l*{4}{c}} 
 \hline \noalign{\smallskip} 
\multicolumn{5}{@{} l}{Dependent variable is Y} \\ 
& \multicolumn{2}{c}{Group 1} & \multicolumn{2}{c}{Group 2}  \\ 
 \hline \noalign{\smallskip} 
\cline{2-3} \cline{4-5}  
 & A & B & C & D \\ 
Var1 & 0.9889809547447591 & 0.5986977239389801 & 0.7023701613078716 & 0.4426388721630653 \\ 
 & 0.620074079120184 & 0.9838676236318822 & 0.7263907806852653 & 0.2614303336710766 \\ 
Var2 & 0.7810378293638999 & 0.5379242479777725 & 0.2756380408586756 & 0.017763905838882765 \\ 
 \hline \noalign{\smallskip} 
 & 0.33152546362433044 & 0.28812168833623875 & 0.1801698611970849 & 0.9664386788512616 \\ 
R2 & 0.5049251235823664 & 0.8328025283692659 & 0.6817932785765529 & 0.5667447260980248 \\ 
\end{tabular*}


In [ ]:
#|export
def make_pdf_from_tex(tex_file_path: Path|str) -> Path:
    """Output PDF is created in the same folder as source tex file. Requires TexLive and its pdflatex utility"""

    if isinstance(tex_file_path,str): tex_file_path = Path(tex_file_path)
    os.chdir(tex_file_path.parent)   
    process = Popen(['pdflatex',tex_file_path], stdin=PIPE, stdout=PIPE, stderr=PIPE,shell=False)
    _ , errors = process.communicate()
    if process.returncode == 0: print("PDF creation successful!")
    else: print("PDF creation failed. Errors:", errors.decode('utf-8'))
    return Path(str.replace(str(tex_file_path), '.tex', '.pdf'))

In [ ]:
#|export
def open_pdf_file(file_path):
    try:
        if platform.system() == "Windows": run(['start', file_path])
        else: run(['open', file_path])
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except SubprocessError:
        print(f"Error opening PDF file: {file_path}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()